In [1]:
import bs4
import requests
import pandas as pd
import numpy as np


In [2]:
Train = pd.read_excel('Data_Train.xlsx')

In [3]:
Test = pd.read_excel('Data_Test.xlsx')

In [4]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
STORY      7628 non-null object
SECTION    7628 non-null int64
dtypes: int64(1), object(1)
memory usage: 119.3+ KB


In [5]:
Train.shape

(7628, 2)

In [6]:
Test.shape

(2748, 1)

In [7]:
df = pd.DataFrame.copy(Train)

In [8]:
df = df.drop_duplicates()


In [9]:
def fix_backslash_n(string):
    string=string.replace('\n',' ')
    #while '  ' in string:
        #string.replace('  ',' ')
    return string

def remove_extra(string):
    if 'This story has been published from a wire agency feed without modifications to the text. Only the headline has been changed.' in string:
        string = string.replace('This story has been published from a wire agency feed without modifications to the text. Only the headline has been changed.',' ')
    if 'This story has been published from a wire agency feed without modifications to the text. Only the headline has been changed' in string:
        string =  string.replace('This story has been published from a wire agency feed without modifications to the text. Only the headline has been changed',' ')
    if 'story has been published from a wire agency feed without modifications to the text. Only the headline has been changed.' in string :
        string = string.replace('story has been published from a wire agency feed without modifications to the text. Only the headline has been changed.',' ')

        
    return string
    
    

df['STORY'] =df['STORY'].apply(fix_backslash_n)
df['STORY'] =df['STORY'].apply(remove_extra)


In [10]:
df = df.drop([5602,112,3744])

In [11]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from num2words import num2words
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [12]:
stop = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone',
             'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount',
             'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around',
             'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before',
             'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both',
             'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de',
             'describe', 'detail', 'did', 'do', 'does', 'doing', 'don', 'done', 'down', 'due', 'during', 'each', 'eg',
             'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone',
             'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for',
             'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had',
             'has', 'hasnt', 'have', 'having', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed',
             'interest', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less',
             'ltd', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly',
             'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine',
             'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once',
             'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
             'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed', 'seeming',
             'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 
             'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system',
             't', 'take', 'ten', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there',
             'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thickv', 'thin', 'third', 'this',
             'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward',
             'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we',
             'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby',
             'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom',
             'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself',
             'yourselves','said','The','As','By','A','In','But','He','She','They','Moreover','We','Its'
             'There','there','However',
             'While','His','With','An','The','This','So','On','Most','If'
       ]

In [13]:
stop_words = stopwords.words('english')
stop_words.extend(stop)
stop = set(stop_words)
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [14]:
def remove_apostrophe(data):
    data = np.char.replace(data,"’","")
    data = np.char.replace(data, "'", "")
    data = np.char.replace(data, "‘", "")
    data = np.char.replace(data, "—", "")
    data = np.char.replace(data,"–", "")
    data = np.char.replace(data,"”","")
    return data

def remove_punctuation(data):
    for i in (exclude):
        data = np.char.replace(data, i, ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    data = np.char.replace(data, "“", "")
    return data

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def clean(data):
    stop_free = " ".join([i for i in data.split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())
    processed = re.sub(r"\d+","",normalized)
    #y = processed.split()
    return processed
def tokenize_and_stem(text):
   
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [15]:
df['STORY'] = df['STORY'].apply(remove_apostrophe)
df['STORY'] = df['STORY'].apply(remove_punctuation)
df['STORY'] = df['STORY'].apply(convert_numbers)
df['STORY'] = df['STORY'].apply(clean)

In [16]:
def get_len(x):
    length = len(x)
    res = len(x.split())
    return res
df['Word_Length'] = df['STORY'].apply(get_len)
df = df[df['Word_Length']>0]

In [17]:
df.dropna(inplace=True)

In [18]:
df = df.drop_duplicates(subset='STORY')

In [19]:
df = df.reset_index(drop=True)
X=df['STORY']
y = df['SECTION']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD


In [54]:
model_adaboost = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer(norm='l2')),
               ('truncatedSVD',TruncatedSVD(algorithm='randomized', n_components=500)),
               ('ada', AdaBoostClassifier(n_estimators=500,
                         learning_rate=1)),
              ])
model_adaboost.fit(X_train,y_train)
model_adaboost_predict = model_adaboost.predict(X_test)

print('accuracy %s' % accuracy_score(model_adaboost_predict, y_test))
print(classification_report(y_test, model_adaboost_predict))

accuracy 0.9480106100795755
             precision    recall  f1-score   support

          0       0.93      0.97      0.95       411
          1       0.95      0.94      0.94       678
          2       0.96      0.94      0.95       488
          3       0.95      0.94      0.95       308

avg / total       0.95      0.95      0.95      1885



In [56]:
model_adaboost = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer(norm='l2')),
               ('truncatedSVD',TruncatedSVD(algorithm='randomized', n_components=350)),
               ('ada', AdaBoostClassifier(n_estimators=350,
                         learning_rate=.1)),
              ])
model_adaboost.fit(X_train,y_train)
model_adaboost_predict = model_adaboost.predict(X_test)

print('accuracy %s' % accuracy_score(model_adaboost_predict, y_test))
print(classification_report(y_test, model_adaboost_predict))

accuracy 0.9374005305039788
             precision    recall  f1-score   support

          0       0.92      0.97      0.95       411
          1       0.93      0.94      0.94       678
          2       0.98      0.90      0.94       488
          3       0.92      0.94      0.93       308

avg / total       0.94      0.94      0.94      1885



In [59]:
ACU = []
for x in range(100,1000,100):
    for y in range(100,1000,100):
        
        model_adaboost = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2),)),
                       ('tfidf', TfidfTransformer(norm='l2')),
                       ('truncatedSVD',TruncatedSVD(algorithm='randomized', n_components=x)),
                       ('ada', AdaBoostClassifier(n_estimators=y,
                                 learning_rate=.1)),
                      ])
        model_adaboost.fit(X_train,y_train)
        model_adaboost_predict = model_adaboost.predict(X_test)

        print('accuracy %s' % accuracy_score(model_adaboost_predict, y_test))
        print(classification_report(y_test, model_adaboost_predict))
        ACU.append((x,y,accuracy_score(model_adaboost_predict, y_test)))


accuracy 0.9257294429708223
             precision    recall  f1-score   support

          0       0.91      0.98      0.94       411
          1       0.91      0.94      0.92       678
          2       0.99      0.89      0.94       488
          3       0.91      0.89      0.90       308

avg / total       0.93      0.93      0.93      1885

accuracy 0.9320954907161804
             precision    recall  f1-score   support

          0       0.91      0.97      0.94       411
          1       0.92      0.94      0.93       678
          2       0.98      0.91      0.95       488
          3       0.91      0.91      0.91       308

avg / total       0.93      0.93      0.93      1885

accuracy 0.9246684350132626
             precision    recall  f1-score   support

          0       0.91      0.98      0.94       411
          1       0.91      0.93      0.92       678
          2       0.98      0.88      0.93       488
          3       0.90      0.91      0.90       308

avg / t

accuracy 0.9326259946949602
             precision    recall  f1-score   support

          0       0.92      0.97      0.95       411
          1       0.92      0.94      0.93       678
          2       0.97      0.89      0.93       488
          3       0.93      0.94      0.93       308

avg / total       0.93      0.93      0.93      1885

accuracy 0.9294429708222812
             precision    recall  f1-score   support

          0       0.91      0.97      0.94       411
          1       0.92      0.94      0.93       678
          2       0.98      0.89      0.93       488
          3       0.91      0.93      0.92       308

avg / total       0.93      0.93      0.93      1885

accuracy 0.9336870026525199
             precision    recall  f1-score   support

          0       0.92      0.97      0.94       411
          1       0.93      0.93      0.93       678
          2       0.96      0.90      0.93       488
          3       0.93      0.93      0.93       308

avg / t

accuracy 0.9342175066312998
             precision    recall  f1-score   support

          0       0.92      0.98      0.94       411
          1       0.93      0.94      0.93       678
          2       0.98      0.90      0.93       488
          3       0.92      0.93      0.92       308

avg / total       0.94      0.93      0.93      1885

accuracy 0.9395225464190982
             precision    recall  f1-score   support

          0       0.92      0.97      0.95       411
          1       0.93      0.95      0.94       678
          2       0.98      0.91      0.95       488
          3       0.93      0.93      0.93       308

avg / total       0.94      0.94      0.94      1885

accuracy 0.9358090185676392
             precision    recall  f1-score   support

          0       0.92      0.98      0.95       411
          1       0.93      0.93      0.93       678
          2       0.97      0.90      0.94       488
          3       0.92      0.94      0.93       308

avg / t

KeyboardInterrupt: 

In [60]:
Acu = pd.Dataframe(ACU)

AttributeError: module 'pandas' has no attribute 'Dataframe'